In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
import random
from pathlib import Path
np.random.seed(42)
random.seed(42)

In [2]:
train_csv = "data_6_channels_train.csv" # ← Pfad ggf. anpassen
df = (pd.read_csv(train_csv)
      .rename(columns={"numerical_id": "forest_id",
                       "class": "is_disturbance",
                       "BLU": "blue", "GRN": "green", "RED": "red",
                       "NIR": "near_infrared",
                       "SW1": "shortwave_infrared_1", "SW2": "shortwave_infrared_2"}))

In [3]:
def engineer_features(group: pd.DataFrame, lags: int = 3) -> pd.DataFrame:
    """Spektrale Indizes + Landsat-8 TCT + 3-Jahres-Stats + Lags/Deltas + Raum/Zeit."""
    eps = 1e-6
    g = group.copy()

    # ---------- Basis-Indizes ----------------------------------------
    g["NDVI"] = (g.near_infrared - g.red) / (g.near_infrared + g.red + eps)
    g["NDMI"] = (g.near_infrared - g.shortwave_infrared_1) / (g.near_infrared + g.shortwave_infrared_1 + eps)
    g["NDWI"] = (g.green - g.near_infrared) / (g.green + g.near_infrared + eps)
    g["NBR"] = (g.near_infrared - g.shortwave_infrared_2) / (g.near_infrared + g.shortwave_infrared_2 + eps)
    g["EVI"] = 2.5 * (g.near_infrared - g.red) / (g.near_infrared + 6 * g.red - 7.5 * g.blue + 1 + eps)
    g["NBR2"] = (g.shortwave_infrared_1 - g.shortwave_infrared_2) / (
                g.shortwave_infrared_1 + g.shortwave_infrared_2 + eps)
    
    # ---------- Trockenheits-Ratio -----------------------------------
    g["SWIR_ratio"] = g.shortwave_infrared_2 / (g.shortwave_infrared_1 + eps)

    # ---------- Landsat-8 Tasseled-Cap (Baig 2014) -------------------
    b2, b3, b4 = g.blue, g.green, g.red
    b5, b6, b7 = g.near_infrared, g.shortwave_infrared_1, g.shortwave_infrared_2
    g["TCB"] = 0.3029 * b2 + 0.2786 * b3 + 0.4733 * b4 + 0.5599 * b5 + 0.5080 * b6 + 0.1872 * b7
    g["TCG"] = -0.2941 * b2 - 0.2430 * b3 - 0.5424 * b4 + 0.7276 * b5 + 0.0713 * b6 - 0.1608 * b7
    g["TCW"] = 0.1511 * b2 + 0.1973 * b3 + 0.3283 * b4 + 0.3407 * b5 - 0.7117 * b6 - 0.4559 * b7
    g["TCT4"] = -0.8239 * b2 + 0.0849 * b3 + 0.4396 * b4 - 0.0580 * b5 + 0.2013 * b6 - 0.2773 * b7
    g["TCT5"] = -0.3294 * b2 + 0.0557 * b3 + 0.1056 * b4 + 0.1855 * b5 - 0.4349 * b6 + 0.8085 * b7
    g["TCT6"] = 0.1079 * b2 - 0.9023 * b3 + 0.4119 * b4 + 0.0575 * b5 - 0.0259 * b6 + 0.0252 * b7

    # --- Interaktions-Features --------------------------------------
    g["TCBxTCG"] = g.TCB * g.TCG
    g["TCBxTCW"] = g.TCB * g.TCW
    g["TCBxNBR"] = g.TCB * g.NBR

    # ---------- 3-Jahres-Median, Std, Anomalie (NDVI & NBR) ----------
    for idx in ("NDVI", "NBR"):
        g[f"{idx}_med3"] = g[idx].rolling(3, min_periods=2).median()
        g[f"{idx}_std3"] = g[idx].rolling(3, min_periods=2).std()
        g[f"{idx}_anom"] = (g[idx] - g[f"{idx}_med3"]) / (g[f"{idx}_med3"].abs() + eps)

    # ---------- Lags & Deltas (blockweise, effizient) ---------------
    base_cols = [
        "NDVI", "NDMI", "NDWI", "NBR", "EVI", "NBR2", "SWIR_ratio",
        "TCB", "TCG", "TCW", "TCT4", "TCT5", "TCT6",
        "TCBxTCG", "TCBxTCW", "TCBxNBR",
        "blue", "green", "red", "near_infrared", "shortwave_infrared_1", "shortwave_infrared_2"
    ]

    lag_features = []
    for lag in range(1, lags + 1):
        shifted = g[base_cols].shift(lag).rename(columns=lambda col: f"l{lag}_{col}")
        deltas = (g[base_cols] - g[base_cols].shift(lag)).rename(columns=lambda col: f"d{lag}_{col}")
        lag_features.extend([shifted, deltas])
    g = pd.concat([g] + lag_features, axis=1)

    # ---------- Raum- & Zeit-Features --------------------------------
    g["year_sin"] = np.sin(2 * np.pi * g.year / 10)
    g["year_cos"] = np.cos(2 * np.pi * g.year / 10)

    return g

In [4]:
#ft_file = "all_data_all_features.csv"
ft_file = "all_data_all_features_fast_approach.csv"
if not Path(ft_file).exists():
    feats = (df.groupby("forest_id")
             .apply(engineer_features)
             .fillna(method="ffill").fillna(method="bfill")
             .reset_index(drop=True))
    feats.to_csv(ft_file, index=False)
else:
    feats = pd.read_csv(ft_file)

feature_cols = [c for c in feats.columns
                if c not in ("forest_id", "year", "is_disturbance")]


In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
correlations = feats.corr(numeric_only=True)["is_disturbance"]
features_to_drop = correlations[correlations.abs() < 0].index.tolist()

#features_to_drop = ['is_disturbance', 'forest_id', 'fid', 'year', 'near_infrared', 'EVI', 'TCB', 'TCT4', 'TCBxTCG', 'NDVI_med3', 'NDVI_anom', 'NBR_med3', 'NBR_anom', 'l1_NDVI', 'l1_NDMI', 'l1_NDWI', 'l1_NBR', 'l1_EVI', 'l1_NBR2', 'l1_SWIR_ratio', 'l1_TCB', 'l1_TCG', 'l1_TCW', 'l1_TCT4', 'l1_TCT5', 'l1_TCT6', 'l1_TCBxTCG', 'l1_TCBxTCW', 'l1_TCBxNBR', 'l1_blue', 'l1_green', 'l1_red', 'l1_near_infrared', 'l1_shortwave_infrared_1', 'l1_shortwave_infrared_2', 'd1_EVI', 'd1_TCT4', 'd1_TCBxTCG', 'd1_near_infrared', 'l2_NDVI', 'l2_NDMI', 'l2_NDWI', 'l2_NBR', 'l2_EVI', 'l2_NBR2', 'l2_SWIR_ratio', 'l2_TCB', 'l2_TCG', 'l2_TCW', 'l2_TCT4', 'l2_TCT5', 'l2_TCT6', 'l2_TCBxTCG', 'l2_TCBxTCW', 'l2_TCBxNBR', 'l2_blue', 'l2_green', 'l2_red', 'l2_near_infrared', 'l2_shortwave_infrared_1', 'l2_shortwave_infrared_2', 'd2_EVI', 'd2_TCT4', 'd2_TCBxTCG', 'd2_near_infrared', 'l3_NDVI', 'l3_NDMI', 'l3_NDWI', 'l3_NBR', 'l3_EVI', 'l3_NBR2', 'l3_SWIR_ratio', 'l3_TCB', 'l3_TCG', 'l3_TCW', 'l3_TCT4', 'l3_TCT5', 'l3_TCT6', 'l3_TCBxTCG', 'l3_TCBxTCW', 'l3_TCBxNBR', 'l3_blue', 'l3_green', 'l3_red', 'l3_near_infrared', 'l3_shortwave_infrared_1', 'l3_shortwave_infrared_2', 'd3_EVI', 'd3_TCT4', 'd3_TCBxTCG', 'd3_near_infrared', 'year_sin', 'year_cos']
feature_cols = [c for c in feats.columns
                if c not in features_to_drop and c not in ('is_disturbance', 'forest_id', 'fid', 'year')]

tr_idx, va_idx = next(gss.split(feats, groups=feats["forest_id"]))

train_df, val_df = feats.iloc[tr_idx], feats.iloc[va_idx]
X_train, y_train = train_df[feature_cols].values, train_df["is_disturbance"].values
X_val, y_val = val_df[feature_cols].values, val_df["is_disturbance"].values
print(f"Train {len(train_df):,} | Val {len(val_df):,}")


Train 362,939 | Val 90,721


In [ ]:
# features_to_drop = ['is_disturbance', 'forest_id', 'fid', 'year', 'near_infrared', 'EVI', 'TCB', 'TCT4', 'TCBxTCG', 'NDVI_med3', 'NDVI_anom', 'NBR_med3', 'NBR_anom', 'l1_NDVI', 'l1_NDMI', 'l1_NDWI', 'l1_NBR', 'l1_EVI', 'l1_NBR2', 'l1_SWIR_ratio', 'l1_TCB', 'l1_TCG', 'l1_TCW', 'l1_TCT4', 'l1_TCT5', 'l1_TCT6', 'l1_TCBxTCG', 'l1_TCBxTCW', 'l1_TCBxNBR', 'l1_blue', 'l1_green', 'l1_red', 'l1_near_infrared', 'l1_shortwave_infrared_1', 'l1_shortwave_infrared_2', 'd1_EVI', 'd1_TCT4', 'd1_TCBxTCG', 'd1_near_infrared', 'l2_NDVI', 'l2_NDMI', 'l2_NDWI', 'l2_NBR', 'l2_EVI', 'l2_NBR2', 'l2_SWIR_ratio', 'l2_TCB', 'l2_TCG', 'l2_TCW', 'l2_TCT4', 'l2_TCT5', 'l2_TCT6', 'l2_TCBxTCG', 'l2_TCBxTCW', 'l2_TCBxNBR', 'l2_blue', 'l2_green', 'l2_red', 'l2_near_infrared', 'l2_shortwave_infrared_1', 'l2_shortwave_infrared_2', 'd2_EVI', 'd2_TCT4', 'd2_TCBxTCG', 'd2_near_infrared', 'l3_NDVI', 'l3_NDMI', 'l3_NDWI', 'l3_NBR', 'l3_EVI', 'l3_NBR2', 'l3_SWIR_ratio', 'l3_TCB', 'l3_TCG', 'l3_TCW', 'l3_TCT4', 'l3_TCT5', 'l3_TCT6', 'l3_TCBxTCG', 'l3_TCBxTCW', 'l3_TCBxNBR', 'l3_blue', 'l3_green', 'l3_red', 'l3_near_infrared', 'l3_shortwave_infrared_1', 'l3_shortwave_infrared_2', 'd3_EVI', 'd3_TCT4', 'd3_TCBxTCG', 'd3_near_infrared', 'year_sin', 'year_cos']

# X_train = train_data.drop(columns=features_to_drop)
# y_train = train_data['is_disturbance']


# X_val = validation_data.drop(columns=features_to_drop)
# y_val = validation_data['is_disturbance']

In [6]:


np.random.seed(42)
# Define the parameter grid for tree sizes
param_grid = {
    'n_estimators': [2000],
    'max_depth': [5],
    'learning_rate': [0.05],
    'subsample': [0.5],
    #'colsample_bytree': [0.6, 0.8, 1.0],
    'scale_pos_weight': [3],
    'gamma': [ 0.3],
    'min_child_weight': [2],
}

search_model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    eta=0.05,
    subsample=0.8,
    colsample_bytree=0.7,
    min_child_weight=2,
    tree_method="hist",
    random_state=42,
    early_stopping_rounds=60
)

group_kfold = GroupKFold(n_splits=3)
# Initialize the GridSearchCV object
grid_search = GridSearchCV(
    estimator=search_model,
    param_grid=param_grid,
    scoring='f1',
    cv=group_kfold,  # This will be replaced by GroupKFold below
    verbose=1,
    n_jobs=-1
)

# To use group-based cross-validation, you need to pass a GroupKFold splitter to the cv argument:


# grid_search.cv = group_kfold.split(X_train, y_train, groups=train_df['forest_id'])

# Perform the grid search
grid_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False, groups=train_df['forest_id'])

# Get the best parameters and the corresponding F1 score
best_params = grid_search.best_params_
best_f1_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best F1 Score: {best_f1_score}")
# Perform prediction on the validation set
y_pred = grid_search.best_estimator_.predict(X_val)

# Calculate F1 score and confusion matrix
f1 = f1_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)

print(f"F1 Score on Validation Set: {f1}")
print(f"Confusion Matrix on Validation Set:\n{cm}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits


Best Parameters: {'gamma': 0.3, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 2000, 'scale_pos_weight': 3, 'subsample': 0.5}
Best F1 Score: 0.5953464148162491
F1 Score on Validation Set: 0.5879396984924623
Confusion Matrix on Validation Set:
[[90159   177]
 [  151   234]]


In [ ]:
features_to_plot = [col for col in X_val.columns if col not in ["year", "forest_id"]]

# Calculate how many rows we need (3 plots per row)
num_features = len(features_to_plot)
num_rows = int(np.ceil(num_features / 3))

# Create subplots with calculated rows and 3 columns
fig, axes = plt.subplots(num_rows, 3, figsize=(18, 6 * num_rows))
axes = axes.flatten()  # Flatten to make indexing easier

# Plot each feature
for i, feature in enumerate(features_to_plot):
    if i < len(axes):  # Make sure we don't exceed the number of available subplots
        ax = axes[i]
        
        # Plot histograms on the current axis
        ax.hist(
            X_val[full_data["is_disturbance"] == 0][feature].dropna(),
            bins=30,
            color='skyblue',
            label='No Disturbance',
            density=True
        )
        ax.hist(
            full_data[full_data["is_disturbance"] == 1][feature].dropna(),
            bins=30,
            color='red',
            label='Disturbance',
            alpha=0.2,
            density=True
        )
        
        # Add labels and title
        ax.set_xlabel(feature)
        ax.set_ylabel("Percentage")
        ax.set_title(f"{feature} distribution")
        ax.grid(False)
        ax.legend()

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the number of neighbors
param_grid_knn = {
    'n_neighbors': list(range(9, 11, 2))  # Odd numbers from 1 to 18
}

# Initialize the KNN classifier
knn_model = KNeighborsClassifier()

# Initialize the GridSearchCV object
grid_search_knn = GridSearchCV(
    estimator=knn_model,
    param_grid=param_grid_knn,
    scoring='f1',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1
)

# Perform the grid search
grid_search_knn.fit(X_train, y_train)

# Get the best parameters and the corresponding F1 score
best_params_knn = grid_search_knn.best_params_
best_f1_score_knn = grid_search_knn.best_score_

print(f"Best Parameters for KNN: {best_params_knn}")
print(f"Best F1 Score for KNN: {best_f1_score_knn}")
# Perform prediction on the validation set using the best KNN model
y_pred_knn = grid_search_knn.best_estimator_.predict(X_val)

# Calculate F1 score and confusion matrix for KNN
f1_knn = f1_score(y_val, y_pred_knn)
cm_knn = confusion_matrix(y_val, y_pred_knn)

print(f"F1 Score on Validation Set for KNN: {f1_knn}")
print(f"Confusion Matrix on Validation Set for KNN:\n{cm_knn}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best Parameters for KNN: {'n_neighbors': 9}
Best F1 Score for KNN: 0.4616615278113527
F1 Score on Validation Set for KNN: 0.4288939051918736
Confusion Matrix on Validation Set for KNN:
[[135418    152]
 [   354    190]]
